
# 🧠 Projeto: Comparador de Jogadores (FBref + Transfermarkt)

Este notebook coleta estatísticas **granulares** do **FBref** (página *All Competitions* de um jogador) e dados biográficos do **Transfermarkt**, agrega **apenas as temporadas selecionadas** (por padrão `2023-2024` e `2024-2025`), cria uma **tabela comparativa** entre dois jogadores com **destaque automático do melhor valor por linha**, gera um **CSV incremental** e oferece **gráficos de radar**.

> **Por que Selenium (stealth headless)?** O FBref costuma bloquear *requests* simples com HTTP 403 e esconde várias tabelas dentro de comentários HTML. Aqui usamos **undetected-chromedriver** para rodar o Chrome em modo headless e aceitamos o consent (cookies) via script. Além disso, "descomentamos" as tabelas antes de parsear.

## Como usar
1. Instale as dependências (célula de instalação abaixo — deixe comentado se já tiver tudo instalado).
2. Execute todas as células até o **Main**.
3. Preencha o nome e as URLs quando solicitado (Transfermarkt & FBref — *All Competitions*).
4. O notebook exibirá a tabela comparativa, salvará/atualizará um CSV (`dataset_coleta_jogadores.csv`) e mostrará radar charts.

## Bibliotecas — o que cada uma faz
- **requests**: baixar HTML do Transfermarkt (e resolver URL canônica de perfil quando a URL não é /profil/).
- **BeautifulSoup (bs4)**: parsear HTML (ambos os sites) e também **descomentar** as tabelas do FBref.
- **pandas**: ler tabelas HTML (via `read_html`), normalizar colunas/linhas e agregar métricas.
- **undetected-chromedriver (Selenium stealth)**: abrir o FBref, aceitar consent, rolar a página e obter o HTML final.
- **numpy**: utilidades numéricas.
- **matplotlib**: gráficos de radar.
- **IPython.display**: exibir tabela estilizada no notebook.
- **re / time / datetime / os**: utilitários diversos.

Alunos: Pedro Henrique Belota Gadelha, Adrian Batista Pereira, Miguel Oliveira Moraes de Souza


## Instalação e Imports

In [13]:

# Se precisar instalar (descomente):
# !pip install undetected-chromedriver beautifulsoup4 lxml pandas numpy matplotlib requests

import os
import re
import time
from io import StringIO
from datetime import datetime, timezone

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, Comment
from IPython.display import display, HTML, clear_output



## Constantes & Configuração Global

- `SEASONS_DEFAULT`: temporadas que entram no cálculo (os DataFrames do FBref já são filtrados na leitura por essas temporadas).
- `FBREF_TABLE_IDS`: mapa de nomes lógicos → lista de possíveis `id` reais no HTML do FBref (as páginas variam).
- `STAT_ALIASES`: alias de colunas por métrica (as colunas mudam levemente por página).
- `PREFERRED_SOURCE`: **fonte única por métrica**, para evitar somar o mesmo indicador duas vezes.
- `INT_METRICS`: métricas que devem ser inteiras (mostradas/gravadas sem casas decimais).


In [14]:

SEASONS_DEFAULT = ['2023-2024', '2024-2025']   # ajuste à vontade

FBREF_TABLE_IDS = {
    "standard":     ["stats_standard_expanded", "stats_standard_combined", "stats_standard"],
    "shooting":     ["stats_shooting_expanded", "stats_shooting_combined", "stats_shooting"],
    "passing":      ["stats_passing_expanded", "stats_passing_combined", "stats_passing"],
    "pass_types":   ["stats_passing_types_expanded", "stats_passing_types_combined", "stats_passing_types"],
    "gca":          ["stats_gca_expanded", "stats_gca_combined", "stats_gca"],
    "defense":      ["stats_defense_expanded", "stats_defense_combined", "stats_defense"],
    "possession":   ["stats_possession_expanded", "stats_possession_combined", "stats_possession"],
    "pressures":    ["stats_pressures_expanded", "stats_pressures_combined", "stats_pressures"],
    "playing_time": ["stats_playing_time_expanded", "stats_playing_time_combined", "stats_playing_time"],
    "misc":         ["stats_misc_expanded", "stats_misc_combined", "stats_misc"],
    "keeper":       ["stats_keeper_expanded", "stats_keeper_combined", "stats_keeper"],
    "keeper_adv":   ["stats_keeper_adv_expanded", "stats_keeper_adv_combined", "stats_keeper_adv"],
}

STAT_ALIASES = {
    "MP": ["MP", "Matches"],
    "Min": ["Min", "Minutes"],
    "Gls": ["Gls", "Goals"],
    "Ast": ["Ast", "Assists"],
    "xG": ["xG"],
    "xAG": ["xAG", "xA"],
    "CrdY": ["CrdY", "Yel"],
    "CrdR": ["CrdR", "Red"],
    "Tkl": ["Tkl", "Tackles"],
    "TklW": ["TklW", "Tackles Won"],
    "Int": ["Int", "Interceptions"],
    "Blocks": ["Blocks"],
    "Clr": ["Clr", "Clearances"],
    "AerialsWon": ["AerialsWon", "Aerials Won"],
    "Pressures": ["Pressures", "Press"],
    "Err": ["Err", "Errors"],
}

PREFERRED_SOURCE = {
    "MP": "standard",
    "Min": "standard",
    "Gls": "standard",
    "Ast": "standard",
    "xG": "shooting",
    "xAG": "passing",
    "CrdY": "standard",
    "CrdR": "standard",
    "Tkl": "defense",
    "TklW": "defense",
    "Int": "defense",
    "Blocks": "defense",
    "Clr": "defense",
    "AerialsWon": "defense",
    "Pressures": "pressures",
    "Err": "misc",
}

INT_METRICS = {
    "MP","Min","Gls","Ast","CrdY","CrdR",
    "Tkl","TklW","Int","Blocks","Clr","AerialsWon","Pressures","Err"
}



## Utilitários (normalização, formatação, helpers)
Inclui correção para `SettingWithCopyWarning` ao limpar as linhas/temporadas.


In [15]:

def is_valid_url(u: str | None) -> bool:
    return bool(re.match(r'^https?://', (u or '').strip()))

def _normalize_season(s: str) -> str:
    return re.sub(r'\s+', '', s or '')

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)
    cols = pd.Index([str(c) for c in df.columns], dtype="object")
    seen = {}
    new_cols = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            new_cols.append(c)
        else:
            seen[c] += 1
            new_cols.append(f"{c}.{seen[c]}")
    df.columns = new_cols
    return df.rename(columns={"Competition": "Comp", "Team": "Squad", "Matches": "MP"})

def _clean_rows(df: pd.DataFrame) -> pd.DataFrame:
    if "Season" in df.columns:
        mask_hdr = df["Season"].astype(str).str.contains(r"Season|years", case=False, na=False)
        df = df.loc[~mask_hdr].copy()
        df.loc[:, "Season"] = df["Season"].astype(str).map(_normalize_season)
    return df.reset_index(drop=True)

def _pick_first_present(df, names):
    for c in names:
        if c in df.columns:
            return c
    return None

def _col_as_series(df: pd.DataFrame, col: str) -> pd.Series:
    obj = df[col]
    return obj.iloc[:, 0] if isinstance(obj, pd.DataFrame) else obj

def _fmt_by_metric(metric: str, value):
    if value is None or value == "":
        return ""
    try:
        fv = float(value)
    except Exception:
        return value
    if metric in INT_METRICS:
        return int(round(fv))
    else:
        return round(fv, 2)



## Scraping Transfermarkt
- Resolve URL canônica de **perfil** quando a URL informada é outra (ex.: `/erfolge/`).
- Extrai **Clube Atual**, **Data de Nascimento**, **Altura**, **Posição**, **Nacionalidade** e **Valor de Mercado**.


In [16]:

def _resolve_transfermarkt_profile_url(url: str) -> str:
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0", "Accept-Language": "pt-BR,pt;q=0.9"}, timeout=30)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        link = soup.find("link", rel="canonical")
        if link and "/profil/spieler/" in (link.get("href") or ""):
            return link["href"]
        a = soup.find("a", href=re.compile(r"/profil/spieler/\d+"))
        if a and a.get("href"):
            href = a["href"]
            return href if href.startswith("http") else "https://www.transfermarkt.com.br" + href
        return url
    except Exception:
        return url

def get_transfermarkt_data(player_url: str) -> dict:
    if not is_valid_url(player_url):
        return {
            "Valor de Mercado": "N/A", "Posição": "N/A", "Nacionalidade": "N/A",
            "Clube Atual": "N/A", "Data de Nascimento": "N/A", "Altura": "N/A"
        }
    profile_url = player_url if "/profil/spieler/" in player_url else _resolve_transfermarkt_profile_url(player_url)
    try:
        r = requests.get(profile_url, headers={"User-Agent": "Mozilla/5.0", "Accept-Language": "pt-BR,pt;q=0.9"}, timeout=30)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        market_value = "N/A"
        mv = soup.select_one("a.data-header__market-value-wrapper")
        if mv:
            txt = mv.get_text(" ", strip=True).replace("\xa0"," ").replace(",",".")
            m = re.search(r"([\d\.]+)\s*(mi\.|mil\.|M|K)", txt, flags=re.I)
            if m:
                num, unit = m.group(1), m.group(2).lower()
                market_value = f"{num}M" if unit in ("mi.","m") else f"{num}K" if unit in ("mil.","k") else num

        dob = height = position = nationality = None
        for li in soup.select(".data-header__details > ul > li"):
            t = li.get_text(" ", strip=True)
            if ("Nasc./Idade" in t) or ("Nasc." in t):
                m = re.search(r"\b(\d{2}/\d{2}/\d{4})\b", t)
                dob = m.group(1) if m else dob
            if "Altura:" in t:
                height = t.split("Altura:")[-1].strip()
            if "Posição:" in t:
                position = t.split("Posição:")[-1].strip()
            if "Nacionalidade:" in t:
                flag = li.find("img")
                nationality = flag["title"] if flag and flag.has_attr("title") else t.split("Nacionalidade:")[-1].strip()

        current_club = None
        label = soup.find("span", string=lambda x: x and "Clube atual" in x)
        if label:
            a = label.find_next("a", title=True)
            current_club = a.get_text(strip=True) if a else None
        if not current_club:
            a = soup.select_one(".data-header__club a[title]")
            if a:
                current_club = a.get_text(strip=True)
        if not current_club:
            a = soup.find("a", href=re.compile(r"/verein/"))
            if a:
                current_club = a.get_text(strip=True)

        return {
            "Valor de Mercado": market_value or "N/A",
            "Posição": position or "N/A",
            "Nacionalidade": nationality or "N/A",
            "Clube Atual": current_club or "N/A",
            "Data de Nascimento": dob or "N/A",
            "Altura": height or "N/A",
        }
    except Exception as e:
        print(f"[Transfermarkt] Erro: {e}")
        return {
            "Valor de Mercado": "Erro", "Posição": "Erro", "Nacionalidade": "Erro",
            "Clube Atual": "Erro", "Data de Nascimento": "Erro", "Altura": "Erro"
        }



## Scraping FBref (Selenium stealth headless)

- Abre a página do jogador (**All Competitions**), aceita o consent de cookies (quando presente), faz *scroll* para carregar tudo, captura o HTML e **descomenta** as tabelas.
- Lê cada tabela usando `pandas.read_html`, **normaliza colunas** e **filtra apenas as `SEASONS_DEFAULT`** já na leitura.


In [17]:

def get_fbref_tables_selenium(url: str, table_id_map: dict, headless: bool = True,
                              scroll_pause: float = 0.7, max_scroll_loops: int = 25) -> dict:
    if not is_valid_url(url):
        return {}

    import undetected_chromedriver as uc
    from selenium.webdriver.support.ui import WebDriverWait

    opts = uc.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=pt-BR")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")

    driver = uc.Chrome(options=opts)

    def _try_accept_consent():
        js = """
        const btns = Array.from(document.querySelectorAll('button, a, input[type="button"], input[type="submit"]'));
        const ok = btns.find(b => /accept|agree|consent|aceitar|concordo/i.test((b.textContent||b.value||'').trim()));
        if (ok) { ok.click(); return true; }
        document.documentElement.style.overflow = 'auto';
        document.body.style.overflow = 'auto';
        return false;
        """
        try:
            return bool(driver.execute_script(js))
        except Exception:
            return False

    try:
        driver.get(url)
        WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")
        _try_accept_consent()
        time.sleep(0.8)

        last_h = 0
        loops = 0
        while True:
            driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
            time.sleep(scroll_pause)
            h = driver.execute_script("return document.body.scrollHeight")
            loops += 1
            if h == last_h or loops >= max_scroll_loops:
                break
            last_h = h

        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "lxml")
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if "<table" in c and "</table>" in c:
            frag = BeautifulSoup(c, "lxml")
            c.replace_with(frag)

    out = {}
    for name, id_variants in table_id_map.items():
        df_ok = None
        for tid in id_variants:
            holder = soup.find(id=f"div_{tid}") or soup.find(id=tid) or soup.find("table", id=tid)
            if not holder:
                continue
            table = holder if holder.name == "table" else holder.find("table", id=tid)
            if not table:
                continue
            try:
                dfs = pd.read_html(StringIO(str(table)))
                if not dfs:
                    continue
                df = dfs[0]
                df = _normalize_columns(df)
                df = _clean_rows(df)
                if "Season" in df.columns:
                    df = df[df["Season"].isin([_normalize_season(x) for x in SEASONS_DEFAULT])]
                if not df.empty:
                    df_ok = df
                    break
            except Exception:
                continue
        if df_ok is not None:
            out[name] = df_ok

    if out:
        print("[FBref] Tabelas capturadas:", {k: v.shape for k, v in out.items()})
    else:
        print("[FBref] Nenhuma tabela encontrada (confira a URL '/all_comps/').")
    return out



## Agregação (sem duplicidade de métricas)
- Para cada métrica, usamos **uma única fonte preferida** (`PREFERRED_SOURCE`).
- Calculamos `G+A_p90` ao final com base em `Min`.


In [18]:

def aggregate_player_stats(fb_tables: dict) -> dict:
    stats = {}
    for metric, src in PREFERRED_SOURCE.items():
        df = fb_tables.get(src)
        if isinstance(df, pd.DataFrame) and not df.empty:
            alias = STAT_ALIASES.get(metric, [metric])
            col = _pick_first_present(df, alias)
            if col:
                s = _col_as_series(df, col)
                stats[metric] = pd.to_numeric(s, errors='coerce').fillna(0).sum()
        else:
            stats[metric] = 0.0

    gols, ast, minutos = stats.get("Gls", 0.0), stats.get("Ast", 0.0), stats.get("Min", 0.0)
    stats["G+A_p90"] = ((gols + ast) / minutos) * 90 if minutos > 0 else 0.0
    return stats



## Visualização — Radar Charts
As funções abaixo normalizam par-a-par entre os dois jogadores e plotam radars **Ofensivo** e **Defensivo**.


In [19]:

def _normalize_pair(a_vals, b_vals):
    out_a, out_b = [], []
    for a, b in zip(a_vals, b_vals):
        a = float(a or 0); b = float(b or 0)
        m = max(a, b)
        out_a.append(a/m if m>0 else 0.0)
        out_b.append(b/m if m>0 else 0.0)
    return out_a, out_b

def _radar_plot(values_a, values_b, labels, name_a, name_b, title):
    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    v1 = list(values_a) + [values_a[0]]
    v2 = list(values_b) + [values_b[0]]
    angles += angles[:1]
    fig, ax = plt.subplots(figsize=(8,8), subplot_kw=dict(polar=True))
    ax.fill(angles, v1, alpha=0.25); ax.plot(angles, v1, linewidth=2, label=name_a)
    ax.fill(angles, v2, alpha=0.25); ax.plot(angles, v2, linewidth=2, label=name_b)
    ax.set_yticklabels([]); ax.set_xticks(angles[:-1]); ax.set_xticklabels(labels, size=12)
    plt.title(title, size=16, y=1.08); plt.legend(loc='upper right', bbox_to_anchor=(1.25, 1.08)); plt.show()

def plot_offensive_radar(stats_a, stats_b, name_a, name_b):
    metrics = ['Gls', 'Ast', 'G+A_p90', 'xG', 'xAG']
    labels  = ['Gols', 'Assist.', 'G+A p/90', 'xG', 'xAG']
    a_vals = [stats_a.get(m, 0) for m in metrics]
    b_vals = [stats_b.get(m, 0) for m in metrics]
    na, nb = _normalize_pair(a_vals, b_vals)
    _radar_plot(na, nb, labels, name_a, name_b, 'Comparativo Ofensivo')

def plot_defensive_radar(stats_a, stats_b, name_a, name_b):
    metrics = ['Tkl', 'Int', 'Blocks', 'Clr', 'AerialsWon', 'Pressures']
    labels  = ['Desarmes','Intercep.','Bloqueios','Rebatidas','Aéreos','Pressões']
    a_vals = [stats_a.get(m, 0) for m in metrics]
    b_vals = [stats_b.get(m, 0) for m in metrics]
    na, nb = _normalize_pair(a_vals, b_vals)
    _radar_plot(na, nb, labels, name_a, name_b, 'Comparativo Defensivo')



## Exportação CSV (2 casas decimais para floats, inteiros sem casas)


In [20]:

def _parse_market_value_to_eur(mv_str: str):
    if not mv_str or mv_str in ("N/A", "Erro"): return None
    s = mv_str.replace("€","").replace(",",".").strip().upper()
    try:
        if s.endswith("M"): return round(float(s[:-1]) * 1_000_000, 2)
        if s.endswith("K"): return round(float(s[:-1]) * 1_000, 2)
        return round(float(s), 2)
    except Exception:
        return None

def _num2(v, default=0.0):
    try:
        return round(float(v), 2)
    except Exception:
        return round(float(default), 2)

def build_players_dataset_rows(pname, qname,
                               p_tm, q_tm,
                               p_stats, q_stats,
                               seasons_list,
                               p_tm_url, p_fb_url,
                               q_tm_url, q_fb_url):
    collected_at = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    seasons = ", ".join(seasons_list)

    def row_for(name, tm, st, tm_url, fb_url):
        return {
            "player": name, "seasons": seasons,
            "source_transfermarkt": tm_url, "source_fbref_all_comps": fb_url,
            "collected_at_utc": collected_at,
            "dob": tm.get("Data de Nascimento"), "height": tm.get("Altura"),
            "current_club": tm.get("Clube Atual"), "nationality": tm.get("Nacionalidade"),
            "position": tm.get("Posição"), "market_value_eur": _parse_market_value_to_eur(tm.get("Valor de Mercado")),
            "matches": _num2(st.get("MP", 0)), "minutes": _num2(st.get("Min", 0)),
            "goals": _num2(st.get("Gls", 0)), "assists": _num2(st.get("Ast", 0)),
            "g_plus_a_per90": _num2(st.get("G+A_p90", 0)),
            "xg": _num2(st.get("xG", 0)), "xag": _num2(st.get("xAG", 0)),
            "yellow_cards": _num2(st.get("CrdY", 0)), "red_cards": _num2(st.get("CrdR", 0)),
            "tackles": _num2(st.get("Tkl", 0)), "interceptions": _num2(st.get("Int", 0)),
            "blocks": _num2(st.get("Blocks", 0)), "clearances": _num2(st.get("Clr", 0)),
            "aerials_won": _num2(st.get("AerialsWon", 0)), "pressures": _num2(st.get("Pressures", 0)),
            "errors": _num2(st.get("Err", 0)),
        }

    rows = [row_for(pname, p_tm, p_stats, p_tm_url, p_fb_url)]
    if qname and (is_valid_url(q_tm_url) or is_valid_url(q_fb_url)):
        rows.append(row_for(qname, q_tm, q_stats, q_tm_url, q_fb_url))
    return rows

def save_dataset_csv(rows, out_path="dataset_coleta_jogadores.csv"):
    df_out = pd.DataFrame(rows)

    int_cols = [
        "matches","minutes","goals","assists","yellow_cards","red_cards",
        "tackles","interceptions","blocks","clearances","aerials_won",
        "pressures","errors"
    ]
    for c in int_cols:
        if c in df_out.columns:
            df_out[c] = pd.to_numeric(df_out[c], errors="coerce").round().astype("Int64")

    float_cols = ["g_plus_a_per90","xg","xag","market_value_eur"]
    for c in float_cols:
        if c in df_out.columns:
            df_out[c] = pd.to_numeric(df_out[c], errors="coerce").round(2)

    file_exists = os.path.exists(out_path)
    df_out.to_csv(out_path, index=False, mode="a", header=not file_exists, encoding="utf-8")
    return os.path.abspath(out_path)



## Execução Principal (inputs → scraping → agregação → tabela/CSV → gráficos)

- Inteiros são exibidos **sem casas**; floats têm **2 casas**.
- A tabela aplica **coloração automática** na coluna do jogador com **melhor valor** por linha.


In [40]:

def main():
    display(HTML("<h2>Comparação — temporadas 2023-2024 e 2024-2025</h2>"))

    p1_name = input("\nNome do jogador 1: ").strip()
    p1_tm_url = input(f"URL Transfermarkt ({p1_name}): ").strip()
    p1_fbref_url = input(f"URL FBref 'All Competitions' ({p1_name}): ").strip()

    p2_name = input("\nNome do jogador 2 (opcional): ").strip()
    p2_tm_url = input(f"URL Transfermarkt ({p2_name}) [opcional]: ").strip() if p2_name else ""
    p2_fbref_url = input(f"URL FBref 'All Competitions' ({p2_name}) [opcional]: ").strip() if p2_name else ""

    single_mode = (not p2_name) or (not is_valid_url(p2_tm_url) and not is_valid_url(p2_fbref_url))
    if single_mode:
        print("\n[Info] Modo 1 jogador.")

    # Transfermarkt
    p1_tm = get_transfermarkt_data(p1_tm_url)
    p2_tm = get_transfermarkt_data(p2_tm_url) if not single_mode else {
        "Valor de Mercado":"N/A","Posição":"N/A","Nacionalidade":"N/A",
        "Clube Atual":"N/A","Data de Nascimento":"N/A","Altura":"N/A"
    }

    # FBref
    print("\n[FBref] Coletando (Selenium stealth headless)...")
    p1_tables = get_fbref_tables_selenium(p1_fbref_url, FBREF_TABLE_IDS, headless=True)
    p2_tables = get_fbref_tables_selenium(p2_fbref_url, FBREF_TABLE_IDS, headless=True) if not single_mode else {}

    # Agregação
    p1_stats = aggregate_player_stats(p1_tables)
    p2_stats = aggregate_player_stats(p2_tables) if not single_mode else {}

    def V(metric, val):
        return _fmt_by_metric(metric, val)

    metricas = [
        "Clube Atual", "Data de Nascimento", "Altura", "Nacionalidade", "Posição", "Valor de Mercado (texto)",
        "Temporadas", "Partidas", "Minutos", "Gols", "Assistências", "G+A p/90", "xG", "xAG",
        "Amarelos", "Vermelhos", "Desarmes", "Intercepções", "Bloqueios", "Rebatidas", "Aéreos ganhos", "Pressões", "Erros"
    ]

    cols = {
        "Métrica": metricas,
        p1_name: [
            p1_tm.get('Clube Atual','-'),
            p1_tm.get('Data de Nascimento','-'),
            p1_tm.get('Altura','-'),
            p1_tm.get('Nacionalidade','-'),
            p1_tm.get('Posição','-'),
            p1_tm.get('Valor de Mercado','-'),
            ", ".join(SEASONS_DEFAULT),
            V("MP", p1_stats.get('MP',0)),
            V("Min", p1_stats.get('Min',0)),
            V("Gls", p1_stats.get('Gls',0)),
            V("Ast", p1_stats.get('Ast',0)),
            V("G+A_p90", p1_stats.get('G+A_p90',0)),
            V("xG", p1_stats.get('xG',0)),
            V("xAG", p1_stats.get('xAG',0)),
            V("CrdY", p1_stats.get('CrdY',0)),
            V("CrdR", p1_stats.get('CrdR',0)),
            V("Tkl", p1_stats.get('Tkl',0)),
            V("Int", p1_stats.get('Int',0)),
            V("Blocks", p1_stats.get('Blocks',0)),
            V("Clr", p1_stats.get('Clr',0)),
            V("AerialsWon", p1_stats.get('AerialsWon',0)),
            V("Pressures", p1_stats.get('Pressures',0)),
            V("Err", p1_stats.get('Err',0)),
        ],
    }

    if not single_mode:
        cols[p2_name] = [
            p2_tm.get('Clube Atual','-'),
            p2_tm.get('Data de Nascimento','-'),
            p2_tm.get('Altura','-'),
            p2_tm.get('Nacionalidade','-'),
            p2_tm.get('Posição','-'),
            p2_tm.get('Valor de Mercado','-'),
            ", ".join(SEASONS_DEFAULT),
            V("MP", p2_stats.get('MP',0)),
            V("Min", p2_stats.get('Min',0)),
            V("Gls", p2_stats.get('Gls',0)),
            V("Ast", p2_stats.get('Ast',0)),
            V("G+A_p90", p2_stats.get('G+A_p90',0)),
            V("xG", p2_stats.get('xG',0)),
            V("xAG", p2_stats.get('xAG',0)),
            V("CrdY", p2_stats.get('CrdY',0)),
            V("CrdR", p2_stats.get('CrdR',0)),
            V("Tkl", p2_stats.get('Tkl',0)),
            V("Int", p2_stats.get('Int',0)),
            V("Blocks", p2_stats.get('Blocks',0)),
            V("Clr", p2_stats.get('Clr',0)),
            V("AerialsWon", p2_stats.get('AerialsWon',0)),
            V("Pressures", p2_stats.get('Pressures',0)),
            V("Err", p2_stats.get('Err',0)),
        ]

    df_final = pd.DataFrame(cols).set_index('Métrica')

    player_cols = [p1_name] if single_mode else [p1_name, p2_name]

    def _highlight_max_row(s: pd.Series):
        nums = pd.to_numeric(s, errors="coerce")
        if nums.isna().all():
            return [''] * len(s)
        is_max = nums == nums.max()
        return ['background-color: #e8f6e8; font-weight: 600' if b else '' for b in is_max]

    styled_df = (df_final.style
                 .apply(_highlight_max_row, axis=1, subset=player_cols)
                 .set_properties(**{'text-align':'center','width':'260px','border':'1px solid #ccc'})
                 .set_table_styles([
                    {'selector':'th','props':[('background-color','#f2f2f2'),('font-size','14px'),('font-weight','bold'),('padding','8px')]},
                    {'selector':'th.row_heading','props':[('text-align','left'),('font-weight','bold'),('white-space','normal')]},
                    {'selector':'tr:nth-child(even)','props':[('background-color','#f9f9f9')]}
                 ])
                 .set_caption(f"<b>COMPARAÇÃO: {p1_name.upper()}{'' if single_mode else ' vs ' + p2_name.upper()}</b>"))

    clear_output(wait=True)
    display(styled_df)

    rows = build_players_dataset_rows(
        p1_name, p2_name if not single_mode else "",
        p1_tm, p2_tm if not single_mode else {},
        p1_stats, p2_stats if not single_mode else {},
        SEASONS_DEFAULT,
        p1_tm_url, p1_fbref_url,
        p2_tm_url if not single_mode else "", p2_fbref_url if not single_mode else ""
    )
    out_csv_path = save_dataset_csv(rows, out_path="dataset_coleta_jogadores.csv")
    print(f"\n[OK] Dataset salvo/atualizado em: {out_csv_path}")

    if not single_mode:
        plot_offensive_radar(p1_stats, p2_stats, p1_name, p2_name)
        plot_defensive_radar(p1_stats, p2_stats, p1_name, p2_name)
    else:
        print("\n[Info] Gráficos requerem 2 jogadores — ignorado em modo 1 jogador.")


main()




Nome do jogador 1:  vini
URL Transfermarkt (vini):  https://www.transfermarkt.pt/vinicius-junior/profil/spieler/371998
URL FBref 'All Competitions' (vini):  https://fbref.com/en/players/7111d552/all_comps/Vinicius-Junior-Stats---All-Competitions

Nome do jogador 2 (opcional):  raphinha
URL Transfermarkt (raphinha) [opcional]:  https://www.transfermarkt.com.br/raphinha/profil/spieler/411295
URL FBref 'All Competitions' (raphinha) [opcional]:  https://fbref.com/en/players/3423f250/all_comps/Raphinha-Stats---All-Competitions



[FBref] Coletando (Selenium stealth headless)...


TypeError: get_fbref_tables_selenium() missing 1 required positional argument: 'table_id_map'

In [25]:
import pandas as pd
import time
from pathlib import Path
from io import StringIO

# Importações do Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def get_league_url_from_user():
    """
    Solicita e valida o link da liga do FBref.
    """
    while True:
        url = input("Digite o link da liga no FBref (ex: https://fbref.com/en/comps/12/2024-2025/stats/...): ")
        if "fbref.com" in url and "/stats/" in url:
            print(f"[OK] Link validado: {url}")
            return url
        else:
            print("\n[Erro] Link inválido. Por favor, insira um link do FBref que contenha '/stats/'. Tente novamente.")

def download_html_with_selenium(url):
    """
    Baixa o conteúdo HTML da URL fornecida usando Selenium para evitar o erro 403.
    """
    print("[INFO] Inicializando o navegador com Selenium para download seguro...")
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        time.sleep(3)
        html_content = driver.page_source
        print(f"[OK] Página baixada com sucesso usando Selenium: {url.split('/')[-1]}")
        return html_content
    except Exception as e:
        print(f"[ERRO CRÍTICO] Falha ao baixar a página com Selenium: {e}")
        return None
    finally:
        if driver:
            driver.quit()

def collect_full_league_data(html_content, min_minutes=450):
    """
    ETAPA 1: Coleta e limpa os dados de TODOS os jogadores, criando nomes de coluna únicos.
    """
    print("\n--- ETAPA 1: Coleta de Dados de Todos os Jogadores ---")
    try:
        html_io = StringIO(html_content)
        df = pd.read_html(html_io, attrs={'id': 'stats_standard'})[0]
    except ValueError:
        print("[Erro] Tabela com id 'stats_standard' não encontrada.")
        return None

    #Lógica para achatar cabeçalhos e garantir nomes únicos.
    new_columns = []
    for col_level1, col_level2 in df.columns:
        if 'Unnamed' in col_level1:
            # Se o nível superior for "Unnamed", usa apenas o nome do nível inferior
            new_columns.append(col_level2)
        else:
            # Caso contrário, combina os dois níveis para criar um nome único e descritivo
            new_columns.append(f"{col_level1}_{col_level2}")
    
    df.columns = new_columns
    
    df = df[df['Player'] != 'Player'].copy()
    
    # Converte colunas para numérico. 
    for col in df.columns:
        if col not in ['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Matches']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df.dropna(subset=['Playing Time_Min'], inplace=True)
    df_filtered = df[df['Playing Time_Min'] >= min_minutes].copy()
    
    print(f"[OK] Coleta concluída. Total de jogadores válidos (≥ {min_minutes} min): {len(df_filtered)}")
    
    filepath = Path.cwd() / "liga_jogadores_completo.csv"
    df_filtered.to_csv(filepath, index=False)
    print(f"[OK] DataFrame completo salvo em: {filepath.resolve()}")
    
    return df_filtered

def calculate_and_save_average(df_full_data):
    """
    ETAPA 2: Recebe o DataFrame completo e calcula a média de todas as colunas numéricas.
    """
    print("\n--- ETAPA 2: Cálculo da Média da Liga ---")
    
    df_numeric = df_full_data.select_dtypes(include='number')
    league_averages = df_numeric.mean()
    
    print("[OK] Média de todas as colunas numéricas calculada.")
    
    filepath = Path.cwd() / "liga_media_liga.csv"
    league_averages.to_frame(name='media').to_csv(filepath)
    print(f"[OK] Arquivo com a média da liga salvo em: {filepath.resolve()}")
    
    return league_averages

def main():
    """
    Orquestra o fluxo: coleta primeiro, calcula depois.
    """
    global df_comparacao
    df_comparacao = pd.DataFrame()

    league_url = get_league_url_from_user()
    html = download_html_with_selenium(league_url)
    
    if html:
        df_liga_completa = collect_full_league_data(html)
        
        if df_liga_completa is not None and not df_liga_completa.empty:
            print("\n[Visualização] 5 primeiras linhas do DataFrame coletado:")
            # Mostra as primeiras 5 linhas e as 10 primeiras colunas para não poluir a tela
            print(df_liga_completa.iloc[:, :10].head())
            
            medias = calculate_and_save_average(df_liga_completa)
            
            df_comparacao = pd.concat([df_comparacao, medias.to_frame(name='Liga (média)').T])
            print("\n[OK] Linha “Liga (média)” adicionada ao dataframe de comparação.")
            
            print("\n--- Resultado Final (DataFrame apenas com a média da liga) ---")
            # Seleciona algumas colunas-chave para uma visualização limpa
            cols_to_show = [
                'Performance_Gls', 'Performance_Ast', 'Performance_G+A',
                'Expected_xG', 'Expected_xAG', 'Playing Time_90s'
            ]
            # Filtra apenas as colunas que realmente existem no dataframe
            existing_cols_to_show = [col for col in cols_to_show if col in df_comparacao.columns]
            print(df_comparacao[existing_cols_to_show].round(2))
        else:
            print("\n[Erro Final] Nenhum dado foi coletado. O processo não pode continuar.")

if __name__ == '__main__':
    main()

Digite o link da liga no FBref (ex: https://fbref.com/en/comps/12/2024-2025/stats/...):  https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats


[OK] Link validado: https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats
[INFO] Inicializando o navegador com Selenium para download seguro...
[OK] Página baixada com sucesso usando Selenium: 2024-2025-La-Liga-Stats

--- ETAPA 1: Coleta de Dados de Todos os Jogadores ---
[OK] Coleta concluída. Total de jogadores válidos (≥ 450 min): 432
[OK] DataFrame completo salvo em: C:\Users\henri\liga_jogadores_completo.csv

[Visualização] 5 primeiras linhas do DataFrame coletado:
    Rk              Player  Nation Pos          Squad  Age  Born  \
3    4         Abdel Abqar  ma MAR  DF         Alavés   25  1999   
5    6              Adrián  es ESP  GK          Betis   37  1987   
6    7  Julen Agirrezabala  es ESP  GK  Athletic Club   23  2000   
7    8       Lucien Agoume  fr FRA  MF        Sevilla   22  2002   
10  11        Nayef Aguerd  ma MAR  DF  Real Sociedad   28  1996   

    Playing Time_MP  Playing Time_Starts  Playing Time_Min  
3                29                   2

In [28]:
import pandas as pd
from IPython.display import display, HTML
from pathlib import Path

def carregar_dados_locais(nome_arquivo="liga_jogadores_completo.csv"):
    """
    Carrega o DataFrame a partir de um arquivo CSV local.
    """
    filepath = Path.cwd() / nome_arquivo
    if not filepath.exists():
        print(f"[Erro] O arquivo '{nome_arquivo}' não foi encontrado.")
        print("Por favor, execute a célula de coleta de dados primeiro para gerar o arquivo.")
        return None
    
    print(f"[OK] Carregando dados do arquivo local: {filepath.resolve()}")
    return pd.read_csv(filepath)

def analisar_medias_por_posicao(df_completo):
    """
    Filtra jogadores por posição, calcula a média e retorna um DataFrame consolidado.
    """
    if df_completo is None or df_completo.empty:
        print("[Erro] O DataFrame de entrada está vazio.")
        return

    print("--- Análise de Médias por Posição ---")
    df = df_completo[df_completo['Playing Time_Min'] >= 450].copy()

    pos_atacantes = ['FW', 'FW,MF']
    pos_meias = ['MF,FW', 'MF']
    pos_defensores = ['DF', 'DF,FW', 'DF,MF']
    pos_goleiros = ['GK']

    df_atacantes = df[df['Pos'].isin(pos_atacantes)]
    df_meias = df[df['Pos'].isin(pos_meias)]
    df_defensores = df[df['Pos'].isin(pos_defensores)]
    df_goleiros = df[df['Pos'].isin(pos_goleiros)]
    
    print(f"Jogadores analisados: {len(df_atacantes)} Atacantes, {len(df_meias)} Meias, {len(df_defensores)} Defensores, {len(df_goleiros)} Goleiros.")

    media_atacantes = df_atacantes.select_dtypes(include='number').mean()
    media_meias = df_meias.select_dtypes(include='number').mean()
    media_defensores = df_defensores.select_dtypes(include='number').mean()
    media_goleiros = df_goleiros.select_dtypes(include='number').mean()

    df_resultado = pd.DataFrame({
        'Média Atacantes': media_atacantes,
        'Média Meias': media_meias,
        'Média Defensores': media_defensores,
        'Média Goleiros': media_goleiros
    }).T

    print("\n[OK] Médias por posição calculadas com sucesso.")
    return df_resultado

#Execução Principal da Análise Local
if __name__ == '__main__':
    df_liga_completa = carregar_dados_locais()
    
    if df_liga_completa is not None:
        df_medias_posicao = analisar_medias_por_posicao(df_liga_completa)

        if df_medias_posicao is not None:
             #Dicionário mapeando nomes antigos para nomes novos
            rename_map = {
                'Performance_Gls': 'Gols',
                'Performance_Ast': 'Assist.',
                'Performance_G+A': 'G+A',
                'Expected_xG': 'xG',
                'Expected_xAG': 'xAG',
                'Playing Time_90s': '90s Jogados',
                'Tackles_Tkl': 'Desarmes'
            }
        
            #Pega apenas as colunas do mapa que existem no DataFrame
            cols_to_show = [col for col in rename_map.keys() if col in df_medias_posicao.columns]
            
            #Cria a visualização apenas com as colunas existentes
            df_view = df_medias_posicao[cols_to_show].round(2)
            
            #Renomeia usando o dicionário, o que é seguro
            df_view = df_view.rename(columns=rename_map)
            
            display(HTML(df_view.to_html(classes='dataframe', border=0)))

[OK] Carregando dados do arquivo local: C:\Users\henri\liga_jogadores_completo.csv
--- Análise de Médias por Posição ---
Jogadores analisados: 101 Atacantes, 124 Meias, 155 Defensores, 33 Goleiros.

[OK] Médias por posição calculadas com sucesso.


,Gols,Assist.,G+A,xG,xAG,90s Jogados
Média Atacantes,5.47,2.37,7.83,5.48,2.44,17.00
Média Meias,2.03,1.85,3.88,2.07,1.96,17.91
Média Defensores,0.68,0.88,1.57,0.77,0.96,19.51
Média Goleiros,0.00,0.06,0.06,0.00,0.06,22.23
